In [1]:
# Import libraries
import pandas as pd
import os

# Define file paths and load data
data_path = "../processed-player-data"
season_file = "combined_data.csv" # Combined dataset
file_path = os.path.join(data_path, season_file)
combined_data = pd.read_csv(file_path)
combined_data # Test successful load

,Player,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards,Season
0,George Gervin,28.0,SAS,SG,82.0,82.0,33.7,10.4,21.1,49.289100,...,3.6,5.1,3.2,1.1,0.7,3.1,2.6,27.1,"MVP-5,AS,NBA1",1980-81
1,Julius Erving,30.0,PHI,SF,82.0,82.0,35.0,9.7,18.6,52.150538,...,5.0,8.0,4.4,2.1,1.8,3.2,2.8,24.6,"MVP-1,AS,NBA1",1980-81
2,Mike Mitchell,25.0,CLE,SF,82.0,82.0,39.0,10.4,21.8,47.706422,...,3.5,6.1,1.7,0.8,0.6,2.1,2.4,24.5,"MVP-24,AS",1980-81
3,Larry Bird,24.0,BOS,PF,82.0,82.0,39.5,8.8,18.3,48.087432,...,8.6,10.9,5.5,2.0,0.8,3.5,2.9,21.2,"MVP-2,AS,NBA1",1980-81
4,Bill Cartwright,23.0,NYK,C,82.0,81.0,35.7,7.5,13.6,55.147059,...,5.5,7.5,1.4,0.6,1.0,2.4,3.2,20.1,No Awards,1980-81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23715,Mac McClung,26.0,ORL,SG,1.0,0.0,5.0,0.0,0.0,0.000000,...,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,No Awards,2024-25
23716,Justin Minaya,25.0,POR,SF,1.0,0.0,6.0,0.0,0.0,0.000000,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,No Awards,2024-25
23717,Riley Minix,24.0,SAS,SF,1.0,0.0,7.0,0.0,1.0,0.000000,...,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,No Awards,2024-25
23718,Jalen Pickett,25.0,DEN,SG,2.0,0.0,2.0,0.0,0.5,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No Awards,2024-25


In [2]:
# Filter players who have exactly '6MOY-1' in the Awards column
smoy_winners = combined_data[combined_data['Awards'] == '6MOY-1']

# Sort by 'MP' (minutes played) and select the lowest 5
lowest_mp_smoy_winners = smoy_winners.sort_values(by='MP').head(5)

# Display relevant columns for these players
lowest_mp_smoy_winners[['Player', 'MP', 'Awards', 'Season']].head()

# Lowest SMOY Winner had 19.3, My cutoff will be 20 'MP'

,Player,MP,Awards,Season
1724,Bill Walton,19.3,6MOY-1,1985-86
9181,Corliss Williamson,21.8,6MOY-1,2001-02
22620,Naz Reid,24.2,6MOY-1,2023-24
16512,Lou Williams,25.2,6MOY-1,2014-15
7100,Danny Manning,25.6,6MOY-1,1997-98


In [3]:
# Filter players who have '6MOY-1' in their Awards column
smoy_players = combined_data[combined_data['Awards'].str.contains(r'6MOY-[1]\b', na=False)]

# Sort by 'G' (Games Played)
top_g_smoy_players = smoy_players[['Player', 'G', 'GS', 'MP', 'Awards', 'Season']].sort_values(by=['G'], ascending=False)

# Display the result
top_g_smoy_players

# Lowest SMOY Winner had 50, My cutoff will be 50 'G'

,Player,G,GS,MP,Awards,Season
10154,Antawn Jamison,82.0,2.0,29.0,6MOY-1,2003-04
4537,Clifford Robinson,82.0,12.0,31.4,6MOY-1,1992-93
14170,Lamar Odom,82.0,35.0,32.2,6MOY-1,2010-11
10744,Ben Gordon,82.0,3.0,24.4,"ROY-2,6MOY-1",2004-05
863,Kevin McHale,82.0,10.0,31.4,"6MOY-1,AS",1983-84
8142,Rodney Rogers,82.0,7.0,27.9,6MOY-1,1999-00
5012,Dell Curry,82.0,0.0,26.5,6MOY-1,1993-94
3686,Detlef Schrempf,82.0,3.0,32.1,6MOY-1,1990-91
5995,Toni Kukoč,81.0,20.0,26.0,6MOY-1,1995-96
22620,Naz Reid,81.0,14.0,24.2,6MOY-1,2023-24


In [4]:
# Filter players who have '6MOY-1' in their Awards column
smoy_players = combined_data[combined_data['Awards'].str.contains(r'6MOY-[1]\b', na=False)]

# Sort by 'GS' (Games Played)
top_gs_smoy_players = smoy_players[['Player', 'G', 'GS', 'MP', 'Awards', 'Season']].sort_values(by=['GS'], ascending=False)

# Display the result
top_gs_smoy_players

# Highest SMOY Winner had 35, My cutoff will be 40 'GS'

,Player,G,GS,MP,Awards,Season
14170,Lamar Odom,82.0,35.0,32.2,6MOY-1,2010-11
8664,Aaron McKie,76.0,33.0,31.5,6MOY-1,2000-01
1200,Kevin McHale,79.0,31.0,33.6,6MOY-1,1984-85
1940,Ricky Pierce,79.0,31.0,31.7,6MOY-1,1986-87
9664,Bobby Jackson,59.0,26.0,28.4,6MOY-1,2002-03
15864,Jamal Crawford,69.0,24.0,30.3,6MOY-1,2013-14
12366,Manu Ginóbili,74.0,23.0,31.1,"MVP-10,6MOY-1,NBA3",2007-08
5995,Toni Kukoč,81.0,20.0,26.0,6MOY-1,1995-96
18287,Lou Williams,79.0,19.0,32.8,6MOY-1,2017-18
11856,Leandro Barbosa,80.0,18.0,32.7,6MOY-1,2006-07


In [5]:
# Save CSV, only including players that fit preliminary SMOY reqs (i.e: MP >= 20, GS <= 40, G >= 50 )
smoy_eligible_data = combined_data[
    (combined_data['GS'] <= 40) &
    (combined_data['MP'] >= 20) &
    (combined_data['G'] >= 50)
]

# Save the filtered data to a new CSV file
output_path = "../processed-player-data/smoy_eligible_players.csv"
smoy_eligible_data.to_csv(output_path, index=False)

# Confirm the data has been saved
print(f"Filtered SMOY data saved to {output_path}")

Filtered SMOY data saved to ../processed-player-data/smoy_eligible_players.csv
